<a href="https://colab.research.google.com/github/agstnhrvy/UPLB-DCE-Thesis/blob/main/Thesis_Code_%5BOrganized4%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries


In [ ]:
# Thesis Project
# To properly install pyswmm and swmm-api to the Python file, select the python interpreter from the venv\Scripts\python.exe then install both libraries using "pip install ..."

!pip install pyswmm
!pip install swmm_api
!pip install swmm5
!pip install openpyxl
import math
import itertools
import os, sys
import swmm5
from pyswmm import Simulation, Links
from pyswmm.swmm5 import SWMMException
from pyswmm.nodes import Nodes
from swmm_api import SwmmInput, SwmmOutput, SwmmReport
from swmm_api.input_file.section_labels import JUNCTIONS, CONDUITS
from swmm_api.input_file.sections import Conduit
from swmm.toolkit.shared_enum import LinkResult
from google.colab import files
import pandas as pd
import re
import copy
import glob

# File Upload

In [ ]:
uploaded = files.upload()

inp = SwmmInput("Study_Model.inp")

# # Update the [RAINGAGES] section if needed
# if "RAINGAGES" in inp:
#     for raingage_name, raingage_data in inp["RAINGAGES"].items():
#         raingage_data.source = "Short Historical Data.prn"  # Set the correct file name

# # Save the modified input file
# inp.write_file("Study_Model_modified.inp")

# out = SwmmOutput("Study_Model.out")
# rpt = SwmmReport("Study_Model.rpt")
available_sizes = uploaded["Available Sizes.txt"]
# ground_slopes = uploaded["Ground_Slopes.xlsx"]
# pipe_prices = uploaded["Pipe_Prices.xlsx"]
print("Uploaded files:", os.listdir())
conduits_df = inp["CONDUITS"]
print(conduits_df)

Saving Available Sizes.txt to Available Sizes.txt
Saving Study_Model.inp to Study_Model.inp
Uploaded files: ['.config', 'Available Sizes.txt', '.ipynb_checkpoints', 'Study_Model.inp', 'sample_data']
{'39': Conduit(name='39', from_node='X21/1', to_node='X21/2', length=19.71, roughness=0.013, offset_upstream=0.0, offset_downstream=0.0, flow_initial=0.0, flow_max=nan), '40': Conduit(name='40', from_node='X23', to_node='X24', length=19.91, roughness=0.013, offset_upstream=0.0, offset_downstream=0.0, flow_initial=0.0, flow_max=nan), '41': Conduit(name='41', from_node='X24', to_node='X25', length=20.0, roughness=0.013, offset_upstream=0.0, offset_downstream=0.0, flow_initial=0.0, flow_max=nan), '42': Conduit(name='42', from_node='X25', to_node='X26', length=17.0, roughness=0.013, offset_upstream=0.0, offset_downstream=0.0, flow_initial=0.0, flow_max=nan), '44': Conduit(name='44', from_node='X1', to_node='X2', length=20.05, roughness=0.013, offset_upstream=0.0, offset_downstream=0.0, flow_ini

# Boundary Parameters

In [ ]:
peak_flow_dict = {}
peak_flow_results =  {}
peak_vel = {}
adjust_pipe_size = {}
pipe_geometry = {}
conduit_ids = []
link_length = {}
in_offset = {}
out_offset = {}
vel_min = 0.8
vel_max = 5
geom1 = {}
geom2 = {}
geom3 = {}
geom4 = {}
trials = {}
velocity = {}

# Data Frame

In [ ]:
conduits = inp['CONDUITS']
xsections = inp['XSECTIONS']
junctions_data = inp['JUNCTIONS']
outfalls_data = inp['OUTFALLS']

# Store results
conduit_info = []

In [ ]:
for junction_name, junction_data in inp["JUNCTIONS"].items():
  junction_dict = vars(junction_data)
  print(junction_dict)

for conduit_name, conduit_data in inp["CONDUITS"].items():
  conduit_dict = vars(conduit_data) # Access all data in Conduit
  print (conduit_dict)

print(xsections)  # Print the content of the xsections InpSection object
for xsection_name, xsection_data in xsections.items():
  print(xsection_data)  # Print the name of each parameter in the xsections section

Test 1

In [ ]:
with Simulation("Study_Model.inp") as sim:
    links = Links(sim)  # Create a Links object
    for step in sim:
      pass
    for link in links:
      conduit_name = link.linkid  # Get conduit name (link ID)
      conduit_data = link.conduit_statistics  # The 'link' object itself holds the conduit data

      peak_flow = conduit_data["peak_flow"]

      # # If you want to store it in a dictionary:
      # peak_flow_dict = {}  # Initialize an empty dictionary if needed
      # peak_flow_dict[conduit_name] = peak_flow
      print(f"Conduit Name: {conduit_name}")
      print(f"Data: {conduit_data}")
      print(f"Peak Flow: {peak_flow}")  # Accessing a property of the conduit
      # Access other properties like link.roughness, link.inlet_offset, etc.

# Data Gathering

In [ ]:
with Simulation("Study_Model.inp") as sim:
    links = Links(sim)
    for step in sim:
      pass
    for link in links:
      conduit_name = link.linkid
      conduit_data = link.conduit_statistics
      peak_flow = conduit_data["peak_flow"]

      print(f"Conduit Name: {conduit_name}")
      print(f"Data: {peak_flow}")

for conduit_name, conduit_data in inp["CONDUITS"].items():
  # Access individual conduit properties using the appropriate keys
  length = conduit_data.length  # Accessing the length property
  in_offset = conduit_data.offset_upstream # parameter for inlet offset
  out_offset = conduit_data.offset_downstream # parameter for outlet offset

  for xsection_name, xsection_data in xsections.items():
    name = xsection_name
    shape = xsection_data.shape
    geom1 = xsection_data.height # height is geom1 for most shapes
    geom2 = xsection_data.parameter_2 # parameter_2 is geom2 for most shapes
    geom3 = xsection_data.parameter_3 # parameter_3 is geom3 for most shapes
    geom4 = xsection_data.parameter_4 # parameter_4 is geom4 for most shapes
    # To get the idea of Geom 1 to Geom 4, see the guide attached in the Code Documentation in Google Docs

    print (f"Name: {name}")
    print (f"Length: {length} m")
    print (f"Shape: {shape}")
    print (f"Geometry Parameters: {geom1} m, {geom2} m, {geom3} m, {geom4} m")
    print (f"Inlet Offset: {in_offset} m")
    print (f"Outlet Offset: {out_offset} m")
    print (f"")

Streaming output truncated to the last 5000 lines.
Outlet Offset: 0.0 m

Name: 352
Length: 20.0 m
Shape: CIRCULAR
Geometry Parameters: 0.91 m, 0.0 m, 0.0 m, 0.0 m
Inlet Offset: 0.0 m
Outlet Offset: 0.0 m

Name: 353
Length: 20.0 m
Shape: CIRCULAR
Geometry Parameters: 0.91 m, 0.0 m, 0.0 m, 0.0 m
Inlet Offset: 0.0 m
Outlet Offset: 0.0 m

Name: 354
Length: 20.0 m
Shape: CIRCULAR
Geometry Parameters: 1.05 m, 0.0 m, 0.0 m, 0.0 m
Inlet Offset: 0.0 m
Outlet Offset: 0.0 m

Name: 355
Length: 20.0 m
Shape: CIRCULAR
Geometry Parameters: 1.05 m, 0.0 m, 0.0 m, 0.0 m
Inlet Offset: 0.0 m
Outlet Offset: 0.0 m

Name: 356
Length: 20.0 m
Shape: CIRCULAR
Geometry Parameters: 1.2 m, 0.0 m, 0.0 m, 0.0 m
Inlet Offset: 0.0 m
Outlet Offset: 0.0 m

Name: 357
Length: 20.0 m
Shape: CIRCULAR
Geometry Parameters: 1.5 m, 0.0 m, 0.0 m, 0.0 m
Inlet Offset: 0.0 m
Outlet Offset: 0.0 m

Name: 358
Length: 20.0 m
Shape: CIRCULAR
Geometry Parameters: 1.5 m, 0.0 m, 0.0 m, 0.0 m
Inlet Offset: 0.0 m
Outlet Offset: 0.0 m

Name: 

# Arrange Conduits based on Pipe Flow


In [ ]:
# Get network topology
conduit_connections = {}
for conduit_name, conduit_data in inp['CONDUITS'].items():
    conduit_connections[conduit_name] = {
        'upstream': conduit_data.from_node,
        'downstream': conduit_data.to_node
    }

# Get all outfall node names from the SWMM input file
outfall_node_names = list(inp['OUTFALLS'].keys())

def is_outfall_node(node):
    """Checks if a node is an outfall node."""
    return node in outfall_node_names  # Direct comparison with outfall node names

def trace_upstream_flow(start_node, path=None):
    """Traces the flow path upstream from a given node, including junctions."""
    if path is None:
        path = []

    path.append(start_node)  # Add the current node (junction or outfall) to the path

    # Find conduits connected to the current node as the downstream
    connected_conduits = [
        conduit_name
        for conduit_name, data in conduit_connections.items()
        if data['downstream'] == start_node
    ]

    # If no upstream conduits are found, it's the outermost junction
    if not connected_conduits:
        return path  # Return the path including the outermost junction

    # Trace upstream for each connected conduit
    for conduit_name in connected_conduits:
        upstream_node = conduit_connections[conduit_name]['upstream']
        new_path = trace_upstream_flow(upstream_node, path + [conduit_name])  # Include conduit in the path
        if new_path:
            return new_path

    return None  # Return None if no path is found (should not happen in a well-defined network)


flow_paths = {}
for outfall_node_name in outfall_node_names:
    flow_path = trace_upstream_flow(outfall_node_name)
    flow_paths[outfall_node_name] = flow_path

# Show results
for outfall_node, flow_path in flow_paths.items():
    flow_path.reverse() # Reverse the flow path list
    print(f"Flow Path for {outfall_node}: {flow_path}")

    # Separate conduits and junctions for clearer output (optional)
    conduits = [item for item in flow_path if item in conduit_connections]
    junctions = [item for item in flow_path if item not in conduit_connections]
    print(f"Sorted Conduits for {outfall_node}: {conduits}")
    print(f"Junctions in Path: {junctions}")

Flow Path for OF-X1: ['X1', '44', 'X2', '45', 'X3', '46', 'X4', '91', 'OF-X1']
Sorted Conduits for OF-X1: ['44', '45', '46', '91']
Junctions in Path: ['X1', 'X2', 'X3', 'X4', 'OF-X1']
Flow Path for OF-X2: ['X21/1', '39', 'X21/2', '386', 'X21', '387', 'X22', '388', 'X23', '40', 'X24', '41', 'X25', '42', 'X26', '43', 'OF-X2']
Sorted Conduits for OF-X2: ['39', '386', '387', '388', '40', '41', '42', '43']
Junctions in Path: ['X21/1', 'X21/2', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'OF-X2']
Flow Path for OF-W1: ['W1', '75', 'W2', '76', 'W3', '77', 'W4', '78', 'W5', '79', 'W6', '80', 'W7', '81', 'W8', '82', 'W9', '83', 'W10', '84', 'W11', '85', 'W12', '86', 'W13', '87', 'W14', '88', 'W15', '89', 'W16', '93', 'OF-W1']
Sorted Conduits for OF-W1: ['75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '93']
Junctions in Path: ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', 'W11', 'W12', 'W13', 'W14', 'W15', 'W16', 'OF-W1']
Flow Path for OF-W2:

# Identify the Invert Elevations of the Conduits

In [ ]:
conduit_info = []
for conduit_name, conduit_data in inp['CONDUITS'].items():
    upstream_junction = conduit_data.from_node
    downstream_junction = conduit_data.to_node

    upstream_invert = junctions_data[upstream_junction].elevation

    # Check if downstream node is an outfall
    if downstream_junction in outfalls_data:
        downstream_invert = outfalls_data[downstream_junction].elevation  # Get elevation from outfalls_data
    elif downstream_junction in junctions_data:
        downstream_invert = junctions_data[downstream_junction].elevation  # Get elevation from junctions_data if it's a junction
    else:
        print(f"Warning: Downstream node '{downstream_junction}' not found in junctions or outfalls data.")
        downstream_invert = None  # or some default value

    length = conduit_data.length

    if downstream_invert is not None:
        slope = (upstream_invert - downstream_invert) / length
    else:
        slope = None
    # Input all data gathered and solve in a DataFrame
    conduit_info.append([conduit_name, upstream_junction, upstream_invert, downstream_junction, downstream_invert, length, slope])

    print(f"Conduit: {conduit_name}; From: {upstream_junction} To: {downstream_junction}")
    print(f"Upstream Invert Elevation: {upstream_invert} m")
    print(f"Downstream Invert Elevation: {downstream_invert} m")
    print(f"Slope: {slope}")

# Exporting DataFrame as an .xlsx File
conduit_df = pd.DataFrame(conduit_info, columns=['Conduit', 'From Node', 'Upstream Invert', 'To Node', 'Downstream Invert', 'Length', 'Slope'])

conduit_df.to_excel("conduit_analysis_data.xlsx", index=False)



Conduit: 39; From: X21/1 To: X21/2
Upstream Invert Elevation: 22.33 m
Downstream Invert Elevation: 22.2 m
Slope: 0.006595636732622983
Conduit: 40; From: X23 To: X24
Upstream Invert Elevation: 21.05 m
Downstream Invert Elevation: 21.01 m
Slope: 0.0020090406830737895
Conduit: 41; From: X24 To: X25
Upstream Invert Elevation: 21.01 m
Downstream Invert Elevation: 20.97 m
Slope: 0.002000000000000135
Conduit: 42; From: X25 To: X26
Upstream Invert Elevation: 20.97 m
Downstream Invert Elevation: 20.93 m
Slope: 0.002352941176470538
Conduit: 44; From: X1 To: X2
Upstream Invert Elevation: 22.33 m
Downstream Invert Elevation: 22.14 m
Slope: 0.009476309226932555
Conduit: 45; From: X2 To: X3
Upstream Invert Elevation: 22.14 m
Downstream Invert Elevation: 22.08 m
Slope: 0.0030000000000001137
Conduit: 46; From: X3 To: X4
Upstream Invert Elevation: 22.08 m
Downstream Invert Elevation: 22.03 m
Slope: 0.0029411764705880683
Conduit: 51; From: T1 To: T2
Upstream Invert Elevation: 22.14 m
Downstream Invert E

Continuity Equation

In [ ]:
# Function for adjusting Pipe Sizes
def adjust_pipe_size (peak_flow, peak_vel):
    Area = peak_flow / peak_vel
    new_diameter = math.sqrt( Area / (0.25 * math.pi))

    pass

# Function: Identifying Peak Flow Rates

In [ ]:
def peak_flow_solve(inp, conduit_name):
  with Simulation(inp) as sim:
      links = Links(sim)
      for step in sim:
          pass

      # Iterate through links to find the specific conduit
      for link in links:
          if link.linkid == conduit_name:
              conduit_data = link.conduit_statistics
              peak_flow = conduit_data.get("peak_flow")  # Use get() to handle missing keys
              if peak_flow is not None and peak_flow != 0.0:
                  return float(peak_flow)  # Convert to float if found
              else:
                  return 0.0  # Or handle the missing peak_flow case differently

      # If the conduit is not found
      return 0.0  # Or handle the missing conduit case differently

Test 1

In [ ]:
def peak_flow(inp_file_or_object):
    if isinstance(inp_file_or_object, str):
        inp_file = inp_file_or_object  # Use the file path directly
    elif isinstance(inp_file_or_object, SwmmInput):
        # If it's a SwmmInput object, write it to a temporary file
        temp_file_name = "temp_input.inp"
        inp_file_or_object.write_file(temp_file_name)
        inp_file = temp_file_name  # Use the temporary file path
    else:
        raise ValueError("Input must be a file path string or a SwmmInput object")

    with Simulation(inp_file) as sim:
        for step in sim:
            pass

        peak_flow_results = {}
        # Assuming 'inp_file_or_object' has an attribute to access CONDUITS data
        # If not, adjust this part to access the conduit data correctly
        for conduit_name, conduit_data in inp_file_or_object["CONDUITS"].items():
            peak_flow_results[conduit_name] = conduit_data.flow_max

    # If a temporary file was created, delete it along with its outputs
    if isinstance(inp_file_or_object, SwmmInput):
        os.remove(temp_file_name)
        os.remove("temp_input.out") # Delete temporary output file
        os.remove("temp_input.rpt") # Delete temporary report file

    return peak_flow

# Function: Identifying Suitable Pipe Sizes

In [ ]:
def find_suitable_sizes(conduit_name, available_sizes, peak_flow):
  if inp["XSECTIONS"][conduit_name].shape == "CIRCULAR":
    original_size = inp["XSECTIONS"][conduit_name].height  # Adjust based on your inp structure

    if peak_flow == 0:
      return [original_size]  # Return original size as the only suitable size

    available_sizes_list = []
    suitable_sizes = []
    with open(available_sizes) as file:
      for line in file:
        size = float(line.strip())
        available_sizes_list.append(float(line.strip()))
        # Changed '^' to '**' for exponentiation
        if peak_flow != 0:  # Add a check for peak_flow
            velocity = peak_flow / (math.pi * size**2 * 0.25)
            if vel_min <= velocity <= vel_max:
              suitable_sizes.append(size)
            elif velocity < vel_min:
              diameter = math.sqrt((peak_flow * 4)/(math.pi * vel_min))
              closest_size = min(available_sizes_list, key=lambda x : abs(x-diameter))
              suitable_sizes.append(closest_size)
            elif velocity > vel_max:
              diameter = math.sqrt((peak_flow * 4)/(math.pi * vel_max))
              closest_size = min(available_sizes_list, key=lambda x : abs(x-diameter))
              suitable_sizes.append(closest_size)
    return suitable_sizes

  else:
    original_size = inp["XSECTIONS"][conduit_name].height
    return [original_size]

# Function: Construction Cost

In [ ]:
# Function for the Computation of Construction Costs
def construction_costs (unit_cost, length, size):
  pass

# Function: Identifying the Largest Pipe Size and Minimum Slope

In [ ]:
# Identify the largest pipe size available and the minimum slope possible
def largest_size_available(available_sizes):
  largest_size = 0.0
  with open(available_sizes) as file:
    for line in file:
      try:
        size = float(line.strip())
        if size > largest_size:
          largest_size = size
      except ValueError:
        print (f"Skipping Invalid Line: {line.strip()}")
  return largest_size

available_sizes = "Available Sizes.txt"
largest_size = largest_size_available(available_sizes)
print(f"Largest Pipe Size: {largest_size} m")

Largest Pipe Size: 5.90551 m


# Function: Solving for the Max Hydraulic Radius

In [ ]:
def get_max_hydraulic_radius(inp, conduit_name):

    with Simulation(inp) as sim:
        links = Links(sim)
        max_hydraulic_radius = 0.0  # Initialize max hydraulic radius

        for step in sim:
            # Iterate through links to find the specific conduit
            for link in links:
                if link.linkid == conduit_name:
                    # Get the flow and depth for the current link and timestep
                    flow = link.flow
                    depth = link.depth

                    # Calculate the hydraulic radius (R = A/P)
                    # Assume circular pipe for this example
                    if inp["XSECTIONS"][conduit_name].shape == "CIRCULAR":
                        diameter = inp["XSECTIONS"][conduit_name].height

                        # Calculate area and wetted perimeter based on depth
                        if depth >= diameter:  # Full flow
                            area = math.pi * (diameter / 2)**2
                            wetted_perimeter = math.pi * diameter
                        else:  # Partial flow
                            theta = 2 * math.acos(1 - (2 * depth / diameter))
                            area = (diameter**2 / 8) * (theta - math.sin(theta))
                            wetted_perimeter = diameter * theta / 2

                        hydraulic_radius = area / wetted_perimeter

                        # Update max_hydraulic_radius if current value is larger
                        max_hydraulic_radius = max(max_hydraulic_radius, hydraulic_radius)

                    else:
                        # Handle other cross-section shapes if needed
                        pass
                    break  # Exit the inner loop once the conduit is found

    return max_hydraulic_radius

# Function: Manning's Equation

In [ ]:
def adjust_slope_with_mannings(input, conduit_name, new_diameter, mannings_n):

    # Load the input file using SwmmInput
    modified_inp = SwmmInput(input_file)

    # Get peak flow rate (Q) for the conduit - you'll need to adapt this part
    # based on how you are getting peak flow rates in your code
    # peak_flow = peak_flow_solve(input_file, conduit_name) # or use your similar function
    # peak_flow = get_peak_flow(modified_inp, conduit_name)
    # peak_flow = current_peak_flows[conduit_name]

    # Calculate cross-sectional area and hydraulic radius for the new diameter
    area = math.pi * (new_diameter / 2)**2
    hydraulic_radius = (new_diameter / 2) / 2  # Assuming full flow for now

    # Calculate the new slope (S) using Manning's equation
    new_slope = (peak_flow * mannings_n / (area * hydraulic_radius**(2/3)))**2

    # Return the modified SwmmInput object
    return new_slope

# Function: Identifying Slopes

In [ ]:
def adjusting_slopes(modified_inp, conduit_name, pipe_size, is_last_conduit):

    # Get upstream and downstream junction names for the conduit
    upstream_junction = modified_inp["CONDUITS"][conduit_name].from_node
    downstream_junction = modified_inp["CONDUITS"][conduit_name].to_node

    # Handle the last conduit differently
    if is_last_conduit:
        # Use original downstream elevation from OUTFALLS section
        new_downstream_elevation = modified_inp["OUTFALLS"][downstream_junction].elevation
    else:
        # Otherwise, get downstream elevation from JUNCTIONS section
        new_downstream_elevation = modified_inp["JUNCTIONS"][downstream_junction].elevation

    # Calculate the new upstream invert elevation using Manning's equation and the optimized size
    new_slope = adjust_slope_with_mannings(modified_inp, conduit_name, pipe_size)
    conduit_length = modified_inp["CONDUITS"][conduit_name].length
    new_upstream_elevation = new_downstream_elevation + (new_slope * conduit_length)

    return new_upstream_elevation

# Step 1: Initialize All Pipe Sizes and Slopes

In [ ]:
for conduit_name, conduit_data in inp["CONDUITS"].items():
    modified_inp = copy.deepcopy(inp)
    for xsection_name, xsection_data in xsections.items():
      # Get the xsection name associated with the current conduit
      # The attribute is called 'xsection' and not 'xsection_name'
      conduit_id = xsection_name

      # Modify the xsection data in the 'XSECTIONS' section
      xsection_data = modified_inp["XSECTIONS"][xsection_name]
      if callable(largest_size_available):
        conduit_size = largest_size_available(available_sizes)
      else:
        conduit_size = largest_size_available

      if xsection_data.shape == "CIRCULAR":
        xsection_data.height = conduit_size
      else:
        pass

      print(f"Name: {conduit_id}")
      print(f"Shape: {xsection_data.shape}")
      print(f"Conduit Height: {xsection_data.height}")
      print(f"")
      # Create a unique filename to avoid overwriting

file_name = f"Modified_Model1.inp"
modified_inp.write_file(file_name)


Streaming output truncated to the last 5000 lines.
Name: 155
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 156
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 157
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 158
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 159
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 160
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 161
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 162
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 163
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 164
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 165
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 166
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 167
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 168
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 169
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 170
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 171
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 172
Shape: CIRCULAR
Conduit Height: 5.90551

Name: 173
Shape: CIRCULAR
Condu

'Modified_Model1.inp'

# Step 2: Optimize Pipe Sizes

Starting Pipe Size Optimization

In [ ]:
# Get all starting pipes from all flow paths
all_starting_pipes = []
for outfall_node, flow_path in flow_paths.items():
    conduits_in_path = [item for item in flow_path if item in conduit_connections]
    all_starting_pipes.append(conduits_in_path[0])

print(all_starting_pipes)

# Create a dictionary to store initial peak flows for each conduit
initial_peak_flows = {}
for conduit_name in all_starting_pipes:  # or inp['CONDUITS'] to check all
    peak_flow_value = peak_flow_solve("Modified_Model1.inp", conduit_name)
    initial_peak_flows[conduit_name] = peak_flow_value
    print(initial_peak_flows)

# Get suitable sizes for all starting pipes
suitable_sizes_per_pipe = {
    conduit_name: find_suitable_sizes(conduit_name, available_sizes, initial_peak_flows[conduit_name])
    for conduit_name in all_starting_pipes
}
print(suitable_sizes_per_pipe)

# Generate all size combinations for all starting pipes
all_size_combinations = list(itertools.product(*[suitable_sizes_per_pipe[pipe] for pipe in all_starting_pipes]))

# Optimization loop for each size combination
for size_combination in all_size_combinations:
    modified_inp = copy.deepcopy(inp)

    # Modify pipe sizes for all starting pipes
    for i, pipe in enumerate(all_starting_pipes):
        # Find the xsection name associated with the pipe
        xsection_name = None
        for xs_name, xs_data in xsections.items():
            if xs_data.link == pipe:
                xsection_name = xs_name
                break

    if xsection_name is not None:
        modified_inp["XSECTIONS"][xsection_name].height = size_combination[i]

    # *** Add this section to save the modified input file ***
    file_name = f"modified_input_starting_pipes_{'_'.join(map(str, size_combination))}.inp"
    modified_inp.write_file(file_name)
    print(f"Modified input file saved as {file_name}")

NameError: name 'flow_paths' is not defined

Downstream Optimization

In [ ]:
# --- Iterate downstream and optimize remaining conduits ---
optimized_input_files = glob.glob("modified_input_starting_pipes_*.inp")

    # Downstream Optimization
for input_file in optimized_input_files:
    modified_inp = SwmmInput(input_file)  # Load input file

    # --- Downstream Optimization Logic ---
    current_peak_flows = {}
    for conduit_name, _ in modified_inp["CONDUITS"].items():
        peak_flow_value = peak_flow_solve(input_file, conduit_name)  # Use input_file here
        current_peak_flows[conduit_name] = peak_flow_value

    for outfall_node, flow_path in flow_paths.items():
        conduits_in_path = [item for item in flow_path if item in conduit_connections]

        # Get suitable sizes for all downstream conduits in the path
        suitable_sizes_per_conduit = {}
        for conduit_name in conduits_in_path:
            # Skip starting pipes as they are already set
            if conduit_name in all_starting_pipes:
                continue
            suitable_sizes_per_conduit[conduit_name] = find_suitable_sizes(conduit_name, available_sizes, current_peak_flows[conduit_name])

        # Generate all size combinations for downstream conduits
        all_size_combinations = list(itertools.product(*[suitable_sizes_per_conduit[pipe] for pipe in suitable_sizes_per_conduit]))

        # Iterate through each size combination
        for size_combination in all_size_combinations:
            combination_modified_inp = copy.deepcopy(modified_inp)  # Create a copy for each combination

            # Apply the size combination to the input file
            for i, conduit_name in enumerate(suitable_sizes_per_conduit):
                # Find the xsection name associated with the conduit
                xsection_name = None
                for xs_name, xs_data in xsections.items():
                    if xs_data.link == conduit_name:
                        xsection_name = xs_name
                        break

                # Update the modified input with the current size from the combination
                if xsection_name is not None:
                    combination_modified_inp["XSECTIONS"][xsection_name].height = size_combination[i]

            # Save the modified input file for this combination
            file_name = f"optimized_{input_file.replace('.inp', '')}_combination_{'_'.join(map(str, size_combination))}.inp"
            combination_modified_inp.write_file(file_name)
            print(f"Optimized input file saved as {file_name}")

# Step 3: Adjust Pipe Slopes Based on Adjusted Pipe Sizes

In [ ]:
# Get a list of all optimized input files from Step 2
optimized_input_files = glob.glob("optimized_modified_input_starting_pipes_*_combination_*.inp")

# Iterate through each optimized input file
for input_file in optimized_input_files:
    # Load the input file using SwmmInput
    modified_inp = SwmmInput(input_file)

    for outfall_node, flow_path in flow_paths.items():
        conduits_in_path = [item for item in flow_path if item in conduit_connections]

        # Process conduits in reverse order (from last to first)
        current_downstream_elevation = None  # Initialize for the first (last) conduit

        for i, conduit_name in enumerate(reversed(conduits_in_path)):
            # Get optimized size
            optimized_size = modified_inp["XSECTIONS"][xsection_name].height

            # Get upstream and downstream junction names
            upstream_junction = modified_inp["CONDUITS"][conduit_name].from_node
            downstream_junction = modified_inp["CONDUITS"][conduit_name].to_node
            roughness = modified_inp["CONDUITS"][conduit_name].roughness
            # If it's the last conduit, use original downstream elevation from OUTFALLS
            if i == 0:  # i = 0 for the last conduit in reversed order
                current_downstream_elevation = modified_inp["OUTFALLS"][downstream_junction].elevation

            # Otherwise, use the upstream elevation of the previous conduit
            # as the current downstream elevation


            # Calculate new slope using Manning's equation and max hydraulic radius
            new_slope = adjust_slope_with_mannings(modified_inp, conduit_name, optimized_size, roughness)

            # Calculate new upstream elevation
            new_upstream_elevation = current_downstream_elevation + (new_slope * modified_inp["CONDUITS"][conduit_name].length)

            # Update upstream junction elevation
            modified_inp["JUNCTIONS"][upstream_junction].elevation = new_upstream_elevation

            # Update current_downstream_elevation for the next conduit in the loop
            current_downstream_elevation = new_upstream_elevation

    # Save the modified input file with a new name
    modified_inp.write_file(f"adjusted_{input_file}")
    print(f"Adjusted slopes/inverts for {input_file} and saved as adjusted_{input_file}")

# Step 4: Optimize Pipe Slopes

In [ ]:
# Define maximum and minimum allowable slopes (adjust values as needed)
max_slope = 0.02  # Maximum allowable slope
min_slope = 0.001  # Minimum allowable slope

# Load ground slope data from "Ground Slopes.xlsx"
ground_slopes_df = pd.read_excel("Ground_Slopes.xlsx")

# Get a list of all adjusted input files from Step 3
adjusted_input_files = [f for f in os.listdir() if f.startswith("adjusted_optimized_input_starting_pipes_")]

# Iterate through each adjusted input file
for input_file in adjusted_input_files:
    # Load the input file using SwmmInput
    modified_inp = SwmmInput(input_file)

    for outfall_node, flow_path in flow_paths.items():
        conduits_in_path = [item for item in flow_path if item in conduit_connections]

        # Process conduits in reverse order (from last to first)
        for i, conduit_name in enumerate(reversed(conduits_in_path)):
            # Get upstream and downstream junction names
            upstream_junction = modified_inp["CONDUITS"][conduit_name].from_node
            downstream_junction = modified_inp["CONDUITS"][conduit_name].to_node

            # Get junction data for upstream and downstream junctions
            upstream_junction_data = modified_inp["JUNCTIONS"][upstream_junction]
            downstream_junction_data = modified_inp["JUNCTIONS"][downstream_junction]

            # Calculate ground slope using junction data
            upstream_elev = upstream_junction_data.elevation + upstream_junction_data.depth_max
            downstream_elev = downstream_junction_data.elevation + downstream_junction_data.depth_max
            length = modified_inp["CONDUITS"][conduit_name].length
            ground_slope = (upstream_elev - downstream_elev) / length

            # Apply slope adjustment logic
            if ground_slope >= 0:
                if ground_slope <= max_slope:
                    final_slope = ground_slope  # Use ground slope if within limits
                else:
                    final_slope = max_slope  # Use max slope if ground slope is too steep
            else:
                final_slope = min_slope  # Use min slope if ground slope is negative

            # If it's the last conduit, use original downstream elevation from OUTFALLS
            if i == 0:
                downstream_elevation = modified_inp["OUTFALLS"][downstream_junction].elevation
            else:
                downstream_elevation = modified_inp["JUNCTIONS"][downstream_junction].elevation

            # Calculate new upstream elevation based on final slope
            upstream_elevation = downstream_elevation + (final_slope * modified_inp["CONDUITS"][conduit_name].length)

            # Update upstream junction elevation in the modified inp file
            modified_inp["JUNCTIONS"][upstream_junction].elevation = upstream_elevation

    # Save the modified input file with a new name
    modified_inp.write_file(f"slope_optimized_{input_file}")
    print(f"Slopes optimized for {input_file} and saved as slope_optimized_{input_file}")

# Step 5: Solving for Construction Costs

In [ ]:
# Load pipe prices data (assuming the file is named "pipe_prices.xlsx")
pipe_prices_df = pd.read_excel("Pipe_Prices.xlsx")

# Get a list of all slope-optimized input files from Step 4
slope_optimized_files = [f for f in os.listdir() if f.startswith("slope_optimized_")]

# Initialize variables to store the least cost and corresponding file name
least_cost = float('inf')  # Initialize with a very large value
least_cost_file = None

# Iterate through each slope-optimized input file
for input_file in slope_optimized_files:
    # Load the input file using SwmmInput
    modified_inp = SwmmInput(input_file)

    total_cost = 0  # Initialize total cost for this input file

    # Iterate through conduits in the input file
    for conduit_name, conduit_data in modified_inp["CONDUITS"].items():
        # Get conduit diameter (assuming it's stored in the 'height' attribute)
        diameter = conduit_data.height

        # Get conduit length
        length = conduit_data.length

        # Get pipe price per length for the current diameter from the pipe_prices_df
        price_per_length = pipe_prices_df[pipe_prices_df["Diameter"] == diameter]["Price per Length"].values[0]

        # Calculate cost for this conduit and add to total cost
        conduit_cost = price_per_length * length
        total_cost += conduit_cost

        # Check if this file has the least cost so far
        if total_cost < least_cost:
          least_cost = total_cost
          least_cost_file = input_file
    # Print total cost for this input file
    print(f"Total construction cost for {input_file}: {total_cost}")

# Print the file with the least cost
print(f"\nFile with the least construction cost: {least_cost_file} (Cost: {least_cost})")

# Sensitivity Analysis


In [ ]:
# Get the file with the least construction cost from Step 5
least_cost_file = "slope_optimized_optimized_modified_input_starting_pipes_0.6_0.45_combination_0.3_0.3_0.45_0.3_0.3_0.3.inp" # Replace with your actual file name

# Load the least cost input file
least_cost_inp = SwmmInput(least_cost_file)

# Function to calculate average flow rate, size, and slope
def get_average_values(inp_file):
    """Calculates average flow rate, size, and slope for all pipes."""
    total_flow = 0
    total_size = 0
    total_slope = 0
    num_pipes = 0

    for conduit_name, conduit_data in inp_file["CONDUITS"].items():
        for xsection_name, xsection_data in inp["XSECTIONS"].items():
        # Calculate slope
          upstream_invert = inp_file["JUNCTIONS"][conduit_data.from_node].elevation
          downstream_invert = inp_file["JUNCTIONS"][conduit_data.to_node].elevation
          length = conduit_data.length
          slope = (upstream_invert - downstream_invert) / length

          # Get peak flow using peak_flow_solve
          peak_flow = peak_flow_solve(inp_file, conduit_name)

          # Get pipe size (assuming it's stored in the 'height' attribute)
          size = conduit_data.height

          total_flow += peak_flow
          total_size += size
          total_slope += slope
          num_pipes += 1

    avg_flow = total_flow / num_pipes if num_pipes else 0
    avg_size = total_size / num_pipes if num_pipes else 0
    avg_slope = total_slope / num_pipes if num_pipes else 0

    return avg_flow, avg_size, avg_slope

# Get average values for the least cost file
avg_flow, avg_size, avg_slope = get_average_values(least_cost_inp)
print(f"Average Flow Rate: {avg_flow}")
print(f"Average Pipe Size: {avg_size}")
print(f"Average Slope: {avg_slope}")

# Choose a pipe for sensitivity analysis (you can modify this)
target_pipe = "C1"  # Replace with the actual pipe name

# Function to get flow rate, size, and slope for a specific pipe
def get_pipe_values(inp_file, pipe_name):
    """Gets flow rate, size, and slope for a specific pipe."""
    conduit_data = inp_file["CONDUITS"][pipe_name]

    # Calculate slope
    upstream_invert = inp_file["JUNCTIONS"][conduit_data.from_node].elevation
    downstream_invert = inp_file["JUNCTIONS"][conduit_data.to_node].elevation
    length = conduit_data.length
    slope = (upstream_invert - downstream_invert) / length

    peak_flow = peak_flow_solve(inp_file, pipe_name) # peak_flow for the chosen conduit
    size = conduit_data.height

    return peak_flow, size, slope

# Get initial values for the target pipe
initial_flow, initial_size, initial_slope = get_pipe_values(least_cost_inp, target_pipe)
print(f"Flow Rate of {target_pipe}: {initial_flow}")
print(f"Size of {target_pipe}: {initial_size}")
print(f"Slope of {target_pipe}: {initial_slope}")


# # Define rainfall scenarios (replace with your actual scenarios)
# rainfall_scenarios = ["baseline", "scenario1", "scenario2"]

# # Sensitivity analysis loop for rainfall scenarios
# for scenario in rainfall_scenarios:
#     # Load modified input file for the current scenario
#     modified_inp_file = f"{scenario}.inp"  # Assuming you have .inp files for each scenario

#     # Create a copy of the least cost input file and modify with the given rainfall intensity
#     modified_inp = copy.deepcopy(least_cost_inp) # modified_inp is now the least cost input

#     # Placeholder - Update rainfall data in the copied least_cost_inp
#     # You need to implement your logic here to update the [TIMESERIES] section in modified_inp with data for the current scenario (modified_inp_file)

#     modified_inp.write_file("modified_least_cost.inp") # write modified inp to a temporary file to check the current peak flows
#     modified_inp = SwmmInput("modified_least_cost.inp") # call this so that the swmm library will update its parameters as well


#     # Get values for the target pipe after applying the rainfall scenario
#     flow, size, slope = get_pipe_values(modified_inp, target_pipe)

#     # Calculate the change in flow rate
#     flow_change = flow - initial_flow

#     # Print the results
#     print(f"\nScenario: {scenario}")
#     print(f"Flow Rate Change for {target_pipe}: {flow_change}")
#     print(f"Flow Rate of {target_pipe}: {flow}")
#     print(f"Size of {target_pipe}: {size}")
#     print(f"Slope of {target_pipe}: {slope}")